# AP(P)3RO PROJECT (model de démo) #


#### M. Le Gras, A. Naullet, A. Calixte, P. Beaunieux ####

## Partie 1: Trouver un Chemin Eulerien ##

On simule une ville avec un petit graphe

In [16]:
import numpy as np

In [17]:
#voici un graph simulant un morceau de ville
#(node1,node2, distance)                                                                                                                                 
G = [(1,2,29), (2,3,13), (3,4,21), (4,8,10), (8,2,2), (8,5,20), (8,6,12), (6,7,6), (6,1,22)]
print(G)

[(1, 2, 29), (2, 3, 13), (3, 4, 21), (4, 8, 10), (8, 2, 2), (8, 5, 20), (8, 6, 12), (6, 7, 6), (6, 1, 22)]


<img src="../assets/img/graph_num.png" width="400">

### 1.2 Transformer un graph non-eulerienen un graph eulerien ###

L'idée c'est de trouver un chemin eulerien, cependant notre graphe n'est pas eulerien
Nous allons alors utilisé un algo qui transforme un graph non-eulerien en un graph eulerien (sans altéré ses propriétés). Enfin on pourra utiliser un algo qui trouve un chemin eulerien c-a-d un chemin qui passe par toutes les arretes du graphes simulant un parcours de toutes les rues de la ville.

In [10]:
# transformation du graph vers un graph eulerien
def odd_vertices(n, edges):
    deg = [0] * n
    for (a,b,c) in edges:
        deg[a] += 1
        deg[b] += 1
        
    return [a for a in range(n) if deg[a] % 2]

def is_edges(n, edges, node1, node2):
    for (a, b, c) in edges:
        if (a == node1 and b == node2) or (a == node2 and b == node1):
            return (a,b,c)
    return None

def list_edges(n, edges, l_vodd):
    res = []
    index = 1
    for a in l_vodd:
        #print("list edges:", (index*100)//len(l_vodd))
        index+=1
        for b in l_vodd:
            if (a == b):
                continue
            tmp = is_edges(n, edges, a, b)
            if (tmp and tmp not in res) :
                #print("MATCH:", a, b)
                res.append(tmp)
    return res

def shortest_edge_idx(l_edges):
    if (len(l_edges) == 0):
        return None
    shortest = l_edges[0][2]
    shortest_index = 0
    
    index = 1
    for (a,b,c) in l_edges[1:]:
        if (c < shortest):
            shortest = c
            shortest_index = index
        index+=1
    return l_edges[shortest_index]

def transform_to_eulerian(n, edges):
    list_vodd = odd_vertices(n, edges)
    l_edges = list_edges(n, edges, list_vodd)
    
    len_vodd = len(list_vodd)
    if (len_vodd == 2 or len_vodd == 0): #case is already eulerian
        return edges
    #print(list_vodd)
    #print(l_edges)
    while(len(list_vodd) != 2):
        list_vodd = odd_vertices(n, edges)
        l_edges = list_edges(n, edges, list_vodd)
        
        shortest_edge = shortest_edge_idx(l_edges)
        #add edge between two
        (a,b,c) = shortest_edge
        new_edge = (b,a,c)
        edges.append(new_edge)
        
        #delete from vodd
        if (a in list_vodd):
            list_vodd.remove(a)
        if (b in list_vodd):
            list_vodd.remove(b)
        
        #delete from list_edges
        l_edges.remove(shortest_edge)
    return edges

 L'algo de transformation en graph eulerien doit donner le graph ci-dessous.<br>
 Le concept est le suivant doubler certaines des arretes des noeuds impaires pour simuler un graphe Eulerien.<br>
 Ainsi, on pourra utiliser un algo qui trouve un chemin eulerien sur notre graphe.

<img src="../assets/img/eulerian_graph.png" width="400">

In [11]:
print("Graph initial:", G)
G_prime = transform_to_eulerian(len(G), G)
print("Graph modifié:", G_prime)

Graph initial: [(1, 2, 29), (2, 3, 13), (3, 4, 21), (4, 8, 10), (8, 2, 2), (8, 5, 20), (8, 6, 12), (6, 7, 6), (6, 1, 22), (7, 6, 6)]
Graph modifié: [(1, 2, 29), (2, 3, 13), (3, 4, 21), (4, 8, 10), (8, 2, 2), (8, 5, 20), (8, 6, 12), (6, 7, 6), (6, 1, 22), (7, 6, 6)]


On a bien une arrête qui s'est rajouté entre les noeuds 6 et 7.<br> Notre algo de transformation vers un graph Eulerien semble fonctionner

### 1.3 Trouver le chemin eulerien ###

In [28]:
from copy import copy

'''
    is_connected - Checks if a graph in the form of a dictionary is 
    connected or not, using Breadth-First Search Algorithm (BFS)
'''
def is_connected(G):
    start_node = list(G)[0]
    color = {v: 'white' for v in G}
    color[start_node] = 'gray'
    S = [start_node]
    while len(S) != 0:
        u = S.pop()
        for v in G[u]:
            if color[v] == 'white':
                color[v] = 'gray'
                S.append(v)
            color[u] = 'black'
    return list(color.values()).count('black') == len(G)

'''
    odd_degree_nodes - returns a list of all G odd degrees nodes
'''
def odd_degree_nodes(G):
    odd_degree_nodes = []
    for u in G:
        if len(G[u]) % 2 != 0:
            odd_degree_nodes.append(u)
    return odd_degree_nodes

'''
    from_dict - return a list of tuples links from a graph G in a 
    dictionary format
'''    
def from_dict(G):
    links = []
    for u in G:
        for v in G[u]:
            links.append((u,v))
    return links

'''
    fleury(G) - return eulerian trail from graph G or a 
    string 'Not Eulerian Graph' if it's not possible to trail a path
'''
def fleury(G):
    '''
        checks if G has eulerian cycle or trail
    '''
    odn = odd_degree_nodes(G)
    #print(odn)
    if len(odn) > 2 or len(odn) == 1:
        return 'Not Eulerian Graph'
    else:
        g = copy(G)
        trail = []
        if len(odn) == 2:
            u = odn[0]
        else:
            u = list(g)[0]
        # - - - 
        init = len(from_dict(g)) + 0.0
        old_pourcentage = -1
        # - - -
        
        while len(from_dict(g)) > 0:
            
            # - - - - Affichage du % - - - - -
            pourcentage = round((init - len(from_dict(g)) )*100.0/init, 1)
            if pourcentage > old_pourcentage:
                len_to_clear = len(str(old_pourcentage))+1
                clear = '\x08' * (len_to_clear + 2)
                old_pourcentage = pourcentage
                print(clear,end="")
                print("\r[*] Compute Eulerian path:", pourcentage, "%", end='', flush=True)
            # - - - - - - - - - - - - - - - - -
            current_vertex = u
            for u in g[current_vertex]:
                g[current_vertex].remove(u)
                g[u].remove(current_vertex)
                bridge = not is_connected(g)
                if bridge:
                    g[current_vertex].append(u)
                    g[u].append(current_vertex)
                else:
                    break
            if bridge:
                g[current_vertex].remove(u)
                g[u].remove(current_vertex)
                g.pop(current_vertex)
            trail.append((current_vertex, u))
    print("")
    return trail

#adapter le format de notre graph au format dictionnaire de l'algo "fleury"
def to_dict(G):
    dict_graph = {}
    for (a,b) in G:
        dict_graph[a] = []
        dict_graph[b] = []
    for (a,b) in G:
        dict_graph[a].append(b)
        dict_graph[b].append(a)
    return dict_graph



In [29]:
#On utlise toutes les fonctions précedement définies:

def transform_and_find_eulerian_path(graph, verbose=False):
    if (verbose):
        print("Initial Graph:", graph)
    graph = transform_to_eulerian(len(graph), graph)
    graph2 = []
    if (verbose):
        print("To Eulerian Graph:", graph)
    ### on enlève la ponderation
    for (a,b,c) in graph:
        graph2.append((a,b))

    dict_graph2 = to_dict(graph2)
    E_path = fleury(dict_graph2)
    if (verbose):
        print("Eulerian path:", E_path)
    return E_path

In [32]:
path = transform_and_find_eulerian_path(G,True)

Initial Graph: [(1, 2, 29), (2, 3, 13), (3, 4, 21), (4, 8, 10), (8, 2, 2), (8, 5, 20), (8, 6, 12), (6, 7, 6), (6, 1, 22), (7, 6, 6)]
To Eulerian Graph: [(1, 2, 29), (2, 3, 13), (3, 4, 21), (4, 8, 10), (8, 2, 2), (8, 5, 20), (8, 6, 12), (6, 7, 6), (6, 1, 22), (7, 6, 6)]
[*] Compute Eulerian path: 90.0 %
Eulerian path: [(2, 1), (1, 6), (6, 7), (7, 6), (6, 8), (8, 4), (4, 3), (3, 2), (2, 8), (8, 5)]


Ce qui donne le chemin suivant:
<img src="../assets/img/eulerian_path.png" width="400">

In [33]:
# TEST en plus
#TEST = [(0,2,12),(2,3,9),(3,0,7),(2,0,23),(2,1,18),(3,1,2),(1,2,4)]
#if (transform_and_find_eulerian_path(TEST)):
#    print("test: OK")

# TEST encore en plus avec un gros graph
tab = [(0, 1, 9), (1, 2, 28), (1, 22, 27), (2, 3, 6), (3, 4, 13), (3, 6, 20), (4, 5, 25), (5, 6, 11), (5, 14, 18), (6, 7, 7), (7, 8, 18), (8, 9, 13), (8, 10, 29), (10, 11, 9), (11, 12, 10), (11, 18, 7), (12, 13, 11), (12, 14, 2), (13, 14, 5), (13, 15, 5), (15, 16, 11), (15, 21, 11), (16, 17, 18), (16, 21, 19), (17, 18, 23), (17, 19, 4), (18, 19, 24), (19, 20, 9), (21, 22, 1)]
len(tab)
big_graph_path = transform_and_find_eulerian_path(tab, True)


###TO REMOVE TEST: 
#tab = [(1, 2, 7.684), (1, 3, 32.165), (1, 4, 47.475), (3, 5, 141.172), (3, 1, 32.165), (3, 6, 225.45000000000002), (5, 3, 141.172), (5, 7, 42.074), (5, 8, 123.21900000000001), (5, 9, 113.902), (10, 11, 49.541), (10, 9, 113.712), (10, 12, 35.389), (13, 14, 66.519), (13, 4, 112.255), (13, 9, 58.559999999999995), (15, 16, 11.34), (15, 17, 35.443999999999996), (15, 18, 11.735), (19, 20, 272.064), (19, 21, 79.89999999999999), (22, 23, 4.266), (22, 24, 208.521), (22, 25, 184.002), (26, 27, 293.0), (28, 29, 90.594), (28, 30, 107.923), (28, 31, 172.778), (28, 32, 198.102), (27, 26, 293.0), (27, 33, 68.338), (27, 20, 111.27900000000001), (8, 34, 20.532), (8, 35, 84.792), (8, 5, 123.21900000000001), (36, 37, 34.317), (36, 38, 75.131), (36, 39, 90.58), (11, 10, 49.541), (11, 40, 42.217), (11, 34, 99.71100000000001), (25, 39, 57.36), (25, 22, 184.00199999999998), (25, 41, 86.86000000000001), (39, 36, 90.58000000000001), (39, 25, 57.36), (39, 41, 94.97099999999999), (39, 38, 78.038), (38, 42, 203.28500000000003), (38, 36, 75.131), (38, 39, 78.038), (43, 44, 19.399), (43, 45, 12.596), (43, 46, 19.651), (44, 47, 10.452), (44, 43, 19.399), (44, 48, 17.821), (47, 49, 5.251), (47, 44, 10.452), (47, 50, 92.137), (49, 47, 5.251), (49, 51, 12.589), (49, 52, 14.724), (45, 43, 12.596), (45, 53, 8.879), (45, 54, 95.439), (55, 23, 86.247), (55, 53, 96.25399999999999), (55, 54, 72.049), (41, 24, 119.134), (41, 39, 94.971), (41, 25, 86.86), (50, 56, 14.274), (50, 47, 92.13699999999999), (50, 48, 74.236), (57, 58, 27.265), (58, 57, 27.265), (58, 59, 33.41), (58, 60, 33.705), (61, 62, 139.41), (61, 63, 33.551), (61, 64, 73.195), (61, 65, 31.847), (61, 66, 10.728000000000002), (67, 68, 3.896), (67, 21, 87.69500000000001), (68, 67, 3.896), (68, 69, 265.00100000000003), (52, 70, 14.852), (52, 51, 18.546999999999997), (52, 49, 14.724), (69, 20, 166.571), (69, 62, 125.478), (69, 68, 265.00100000000003), (69, 70, 311.439), (20, 19, 272.064), (20, 31, 243.596), (20, 27, 111.279), (20, 69, 166.571), (71, 46, 130.171), (71, 72, 135.094), (71, 73, 168.456), (71, 74, 67.595), (46, 71, 130.171), (46, 43, 19.651), (46, 75, 108.99600000000001), (76, 77, 52.769), (76, 78, 82.916), (76, 79, 175.67699999999996), (76, 80, 77.902), (81, 82, 5.048), (81, 83, 19.891), (81, 75, 115.99300000000001), (81, 84, 16.95), (84, 85, 94.196), (84, 73, 25.564), (84, 81, 16.95), (72, 71, 135.094), (72, 86, 99.995), (72, 87, 191.63199999999998), (88, 89, 9.273), (88, 90, 31.682000000000002), (88, 91, 116.40899999999999), (86, 89, 55.587), (86, 92, 145.93800000000002), (86, 93, 91.836), (86, 72, 99.995), (94, 92, 83.758), (94, 95, 118.39699999999999), (94, 96, 80.912), (94, 97, 135.121), (92, 74, 36.945), (92, 94, 83.758), (92, 86, 145.93800000000002), (92, 56, 203.07799999999997), (74, 71, 67.595), (74, 92, 36.945), (74, 48, 163.477), (73, 71, 168.456), (73, 87, 48.187), (73, 84, 25.564), (48, 50, 74.236), (48, 44, 17.821), (48, 74, 163.47700000000003), (56, 50, 14.274), (56, 98, 18.632), (56, 92, 203.078), (70, 52, 14.852), (70, 99, 74.472), (70, 69, 311.43899999999996), (99, 70, 74.472), (93, 96, 70.578), (93, 100, 151.787), (93, 86, 91.836), (101, 95, 126.203), (101, 65, 35.254), (101, 98, 72.322), (101, 63, 34.226), (101, 102, 11.685), (103, 104, 122.99199999999999), (103, 105, 157.89399999999998), (103, 106, 152.29099999999997), (91, 88, 116.409), (107, 104, 80.576), (104, 90, 143.1), (104, 103, 122.99199999999999), (104, 107, 80.57600000000001), (108, 89, 34.944), (89, 88, 9.273), (89, 86, 55.587), (89, 108, 34.944), (109, 106, 48.785), (109, 110, 17.728), (109, 100, 24.491), (111, 96, 134.828), (96, 94, 80.912), (96, 93, 70.578), (96, 111, 134.828), (110, 112, 171.75400000000002), (110, 113, 130.934), (110, 109, 17.728), (114, 100, 66.318), (100, 114, 66.318), (100, 109, 24.491), (100, 93, 151.787), (115, 106, 88.78399999999999), (106, 109, 48.785), (106, 103, 152.291), (106, 115, 88.784), (95, 101, 126.203), (95, 94, 118.397), (95, 116, 123.33899999999998), (117, 97, 57.61), (97, 117, 57.61), (97, 94, 135.121), (97, 112, 57.211), (116, 95, 123.339), (116, 118, 122.207), (116, 119, 53.465999999999994), (119, 116, 53.466), (120, 113, 187.381), (120, 118, 155.192), (120, 112, 84.11000000000001), (120, 121, 119.39099999999999), (113, 122, 74.508), (113, 120, 187.38099999999997), (113, 77, 35.869), (113, 110, 130.934), (123, 124, 64.055), (123, 122, 102.161), (123, 79, 97.856), (105, 125, 6.272), (105, 103, 157.894), (105, 124, 20.93), (118, 126, 18.834), (118, 116, 122.207), (118, 120, 155.192), (79, 127, 43.034), (79, 76, 175.677), (79, 123, 97.856), (124, 123, 64.055), (124, 105, 20.93), (124, 128, 372.629), (78, 128, 33.936), (78, 129, 51.602000000000004), (78, 76, 82.91600000000001), (122, 123, 102.161), (122, 113, 74.508), (122, 77, 113.615), (77, 76, 52.769), (77, 122, 113.61500000000001), (77, 113, 35.869), (80, 130, 70.251), (80, 130, 264.6), (80, 76, 77.902), (121, 131, 44.765), (121, 130, 33.822), (121, 120, 119.39099999999999), (132, 133, 137.26), (132, 60, 11.13), (132, 134, 16.81), (133, 135, 134.543), (133, 135, 134.542), (133, 132, 137.26), (133, 136, 11.242), (135, 137, 3.534), (135, 133, 134.543), (135, 133, 134.542), (137, 135, 3.534), (137, 126, 33.143), (137, 138, 121.037), (131, 139, 22.379), (131, 121, 44.765), (131, 140, 132.825), (141, 139, 135.471), (139, 142, 53.52), (139, 131, 22.379), (139, 141, 135.471), (143, 142, 189.75900000000001), (142, 139, 53.52), (142, 136, 125.272), (142, 143, 189.759), (62, 69, 125.47800000000001), (62, 61, 139.41), (21, 19, 79.89999999999999), (21, 67, 87.69500000000001), (31, 20, 243.596), (31, 144, 119.346), (31, 28, 172.77800000000002), (145, 146, 20.004), (145, 147, 11.114), (145, 148, 13.059999999999999), (148, 149, 16.493), (148, 150, 11.545000000000002), (148, 145, 13.059999999999999), (151, 152, 13.106), (151, 153, 7.018000000000001), (151, 154, 11.608), (146, 145, 20.004), (146, 155, 11.687999999999999), (146, 156, 14.112), (157, 152, 34.609), (157, 158, 67.752), (157, 159, 34.306), (160, 161, 37.727), (160, 162, 21.16), (160, 16, 39.001), (154, 159, 13.907), (154, 163, 40.548), (154, 151, 11.608), (164, 18, 4.275), (164, 16, 6.706), (164, 162, 16.32), (165, 166, 51.077), (165, 167, 15.273), (165, 168, 18.264), (169, 170, 6.97), (169, 171, 28.194000000000003), (169, 172, 23.836), (169, 173, 13.706), (174, 175, 44.852999999999994), (174, 176, 14.344999999999999), (174, 177, 83.283), (174, 173, 61.202), (178, 179, 20.995), (178, 180, 21.35), (178, 177, 85.149), (168, 181, 11.451), (168, 167, 9.503), (168, 165, 18.264), (182, 183, 6.746), (182, 184, 14.24), (182, 185, 35.290000000000006), (167, 165, 15.273), (167, 168, 9.503), (167, 17, 11.648), (186, 187, 17.559), (186, 188, 6.229), (186, 189, 18.953000000000003), (186, 190, 10.373000000000001), (187, 191, 16.867), (187, 186, 17.559), (187, 188, 15.602), (191, 187, 16.867), (191, 179, 6.945), (191, 192, 41.135), (189, 193, 5.041), (189, 194, 38.048), (189, 186, 18.953000000000003), (189, 190, 9.289), (195, 2, 9.812), (195, 196, 7.688), (195, 197, 11.807), (195, 198, 150.965), (199, 200, 9.275), (199, 201, 6.362), (199, 202, 18.055), (199, 4, 14.408000000000001), (203, 204, 6.762), (203, 205, 120.768), (203, 206, 34.564), (206, 207, 10.492), (206, 208, 6.904), (206, 203, 34.564), (208, 209, 6.34), (208, 206, 6.904), (208, 210, 11.335), (208, 204, 26.169), (193, 189, 5.041), (193, 211, 7.606), (193, 212, 25.103), (193, 194, 36.99), (213, 214, 4.232), (213, 215, 5.38), (213, 216, 5.221), (213, 217, 31.358), (218, 219, 7.03), (218, 220, 4.946), (218, 221, 11.744), (218, 207, 7.984), (218, 222, 10.024999999999999), (219, 218, 7.03), (219, 223, 8.304), (219, 224, 28.674999999999997), (223, 219, 8.304), (223, 207, 6.685), (223, 215, 28.504), (221, 225, 4.101), (221, 218, 11.744), (221, 226, 28.976), (227, 228, 29.733), (227, 229, 9.121), (227, 230, 175.889), (227, 231, 37.56), (228, 227, 29.733), (228, 232, 8.168), (228, 233, 99.757), (228, 231, 36.759), (233, 234, 37.004), (233, 228, 99.757), (233, 235, 186.699), (233, 236, 8.744), (237, 234, 37.256), (237, 238, 115.562), (237, 239, 17.684), (240, 241, 8.083), (240, 242, 231.981), (240, 243, 6.852), (240, 244, 6.102), (245, 246, 7.284), (245, 247, 5.822), (245, 248, 232.14700000000002), (245, 249, 10.057), (250, 251, 82.09200000000001), (250, 252, 68.713), (250, 253, 234.41200000000003), (4, 254, 6.634), (4, 1, 47.474999999999994), (4, 199, 14.408000000000001), (4, 13, 112.255), (255, 256, 52.18), (255, 254, 48.264), (255, 257, 10.26), (256, 255, 52.18), (256, 183, 12.022), (256, 192, 65.226), (256, 14, 79.559), (192, 190, 4.487), (192, 256, 65.226), (192, 191, 41.135000000000005), (258, 259, 34.351), (258, 184, 11.747), (258, 260, 47.373), (259, 261, 11.271), (259, 258, 34.351), (259, 262, 68.618), (262, 263, 13.696), (262, 264, 7.525), (262, 259, 68.618), (265, 266, 7.602), (265, 267, 9.351), (265, 260, 37.646), (266, 265, 7.602), (266, 204, 38.489999999999995), (266, 210, 12.353), (268, 267, 14.762), (268, 269, 17.425), (268, 220, 9.543), (268, 212, 8.821), (268, 209, 6.631), (268, 222, 4.02), (270, 238, 62.236999999999995), (270, 271, 34.578), (270, 272, 28.659000000000002), (271, 273, 24.719), (271, 270, 34.578), (271, 274, 117.32400000000001), (275, 276, 17.102), (275, 273, 32.449), (275, 277, 243.149), (276, 278, 145.637), (276, 279, 13.842), (276, 275, 17.102), (279, 280, 134.018), (279, 281, 24.279), (279, 276, 13.842), (281, 282, 177.212), (281, 283, 25.335), (281, 279, 24.279), (283, 284, 17.663), (283, 281, 25.335), (283, 285, 112.045), (284, 272, 7.12), (284, 286, 58.151999999999994), (284, 283, 17.663), (287, 288, 25.253), (287, 289, 130.095), (287, 290, 66.574), (291, 292, 3.947), (291, 293, 32.399), (291, 294, 131.63), (260, 292, 182.61200000000002), (260, 185, 13.689), (260, 258, 47.373), (260, 265, 37.646), (252, 295, 158.105), (252, 250, 68.713), (252, 296, 70.601), (297, 298, 89.694), (297, 299, 154.64000000000001), (297, 300, 50.024), (301, 296, 119.095), (301, 302, 190.81799999999998), (301, 303, 64.866), (299, 293, 141.502), (299, 304, 53.295), (299, 297, 154.64), (304, 305, 4.151), (304, 299, 53.294999999999995), (304, 306, 18.65), (307, 308, 35.635), (307, 305, 30.686), (307, 306, 47.197), (308, 307, 35.635), (308, 197, 13.392), (308, 264, 19.043999999999997), (197, 195, 11.807), (197, 264, 12.176), (197, 308, 13.392), (197, 196, 10.198), (264, 200, 14.381), (264, 262, 7.525), (264, 197, 12.176), (264, 308, 19.043999999999997), (309, 310, 95.809), (309, 302, 81.621), (309, 311, 61.153999999999996), (311, 312, 72.58), (311, 313, 54.213), (311, 309, 61.153999999999996), (313, 314, 37.84), (313, 311, 54.213), (313, 315, 188.37800000000001), (315, 289, 149.091), (315, 316, 40.931999999999995), (315, 313, 188.37800000000001), (294, 317, 40.772999999999996), (294, 291, 131.63), (294, 318, 34.589), (294, 289, 160.633), (298, 318, 1.687), (298, 319, 38.578), (298, 297, 89.694), (295, 316, 84.982), (295, 252, 158.105), (295, 312, 192.681), (312, 311, 72.58), (312, 302, 42.379), (312, 295, 192.681), (302, 309, 81.621), (302, 301, 190.81799999999998), (302, 312, 42.379), (289, 287, 130.095), (289, 315, 149.091), (289, 294, 160.633), (310, 320, 27.1), (310, 309, 95.809), (310, 314, 88.684), (314, 313, 37.84), (314, 321, 32.134), (314, 310, 88.684), (296, 252, 70.601), (296, 301, 119.095), (296, 322, 38.339), (322, 296, 38.339), (317, 323, 15.758), (317, 294, 40.772999999999996), (317, 319, 35.201), (300, 320, 29.423), (300, 319, 75.473), (300, 297, 50.024), (319, 317, 35.201), (319, 300, 75.473), (319, 298, 38.578), (293, 291, 32.399), (293, 299, 141.502), (293, 318, 134.463), (9, 10, 113.712), (9, 5, 113.902), (9, 13, 58.559999999999995), (9, 324, 47.620999999999995), (177, 178, 85.149), (177, 174, 83.283), (177, 325, 13.555), (7, 6, 73.17), (7, 5, 42.074), (7, 326, 131.011), (324, 14, 67.397), (324, 325, 45.66), (324, 9, 47.620999999999995), (176, 12, 36.236), (176, 174, 14.344999999999999), (176, 325, 88.668), (325, 176, 88.668), (325, 177, 13.555), (325, 324, 45.66), (6, 7, 73.17), (6, 3, 225.45000000000002), (6, 37, 61.248), (327, 328, 10.312), (327, 329, 59.976), (327, 330, 14.581999999999999), (331, 332, 50.106), (331, 331, 120.12399999999998), (331, 331, 120.124), (333, 334, 52.588), (333, 335, 37.866), (333, 329, 34.444), (158, 161, 52.531), (158, 157, 67.752), (158, 334, 75.808), (35, 8, 84.792), (35, 37, 57.084), (35, 336, 79.645), (336, 337, 22.501), (336, 35, 79.645), (336, 338, 85.918), (337, 338, 54.559), (337, 336, 22.501), (337, 339, 158.55), (14, 13, 66.519), (14, 324, 67.39699999999999), (14, 256, 79.559), (172, 340, 34.619), (172, 169, 23.836), (172, 341, 17.664), (342, 343, 23.313000000000002), (342, 344, 127.162), (342, 345, 27.91), (344, 346, 56.276), (344, 347, 119.922), (344, 342, 127.162), (346, 344, 56.276), (346, 348, 230.12900000000005), (346, 349, 58.893), (349, 350, 55.647), (349, 346, 58.893), (349, 235, 122.114), (351, 350, 6.729), (351, 352, 210.209), (351, 353, 92.56799999999998), (353, 348, 80.28), (353, 354, 142.978), (353, 351, 92.56800000000001), (354, 355, 169.916), (354, 353, 142.978), (354, 356, 188.83599999999998), (354, 352, 62.387), (357, 358, 394.367), (357, 359, 16.242), (357, 360, 12.145), (361, 362, 24.48), (361, 360, 16.837), (361, 359, 15.248999999999999), (360, 363, 26.355), (360, 357, 12.145), (360, 361, 16.837), (348, 346, 230.12900000000002), (348, 356, 313.99699999999996), (348, 353, 80.28), (348, 364, 115.54899999999999), (149, 148, 16.493), (149, 365, 455.166), (149, 150, 13.542000000000002), (365, 356, 78.581), (365, 149, 455.16600000000005), (365, 366, 160.66), (356, 365, 78.581), (356, 348, 313.997), (356, 364, 354.27700000000004), (356, 367, 103.92999999999999), (356, 354, 188.836), (356, 366, 152.93800000000002), (368, 369, 57.266), (368, 370, 317.885), (368, 371, 273.976), (368, 372, 279.315), (372, 373, 226.566), (372, 368, 279.315), (372, 374, 48.150999999999996), (375, 376, 73.353), (375, 377, 112.381), (375, 378, 12.82), (371, 379, 193.376), (371, 368, 273.97600000000006), (371, 376, 243.40099999999998), (366, 377, 108.991), (366, 363, 166.431), (366, 365, 160.66000000000003), (366, 356, 152.93800000000002), (364, 348, 115.549), (364, 356, 354.277), (364, 380, 130.414), (381, 382, 52.592), (381, 383, 321.034), (381, 384, 63.767), (381, 385, 229.178), (386, 362, 83.019), (386, 355, 120.322), (386, 387, 100.426), (386, 388, 71.745), (367, 355, 81.134), (367, 356, 103.92999999999999), (367, 389, 102.537), (347, 390, 60.723), (347, 235, 118.28800000000001), (347, 344, 119.922), (347, 231, 153.331), (235, 347, 118.288), (235, 233, 186.69899999999998), (235, 391, 117.34100000000001), (235, 349, 122.114), (391, 392, 76.78), (391, 393, 146.48600000000002), (391, 235, 117.34100000000001), (231, 227, 37.559999999999995), (231, 347, 153.331), (231, 228, 36.759), (394, 395, 13.298), (394, 396, 140.45), (394, 397, 24.591), (394, 398, 6.352), (399, 400, 119.74300000000001), (399, 401, 89.187), (399, 402, 26.216), (399, 403, 106.634), (402, 404, 158.757), (402, 405, 92.291), (402, 399, 26.216), (402, 406, 96.013), (407, 408, 13.218), (407, 409, 17.697000000000003), (407, 410, 151.945), (409, 411, 147.811), (409, 412, 13.866), (409, 407, 17.697000000000003), (412, 413, 15.86), (412, 409, 13.866), (412, 414, 144.33), (413, 415, 141.824), (413, 416, 15.526), (413, 412, 15.86), (416, 417, 139.828), (416, 418, 17.078000000000003), (416, 413, 15.526), (419, 420, 147.991), (419, 395, 19.101), (419, 421, 16.072000000000003), (422, 423, 25.191), (422, 424, 6.471), (422, 425, 231.503), (422, 426, 15.184000000000001), (427, 428, 11.219), (427, 429, 89.78), (427, 426, 44.467000000000006), (428, 423, 59.468), (428, 427, 11.219), (428, 430, 19.383000000000003), (431, 432, 83.222), (431, 430, 2.559), (431, 433, 56.864000000000004), (433, 434, 196.815), (433, 435, 11.612), (433, 431, 56.864), (435, 248, 235.68), (435, 436, 25.484), (435, 433, 11.612), (435, 437, 5.729), (408, 407, 13.218), (408, 438, 26.085), (408, 439, 6.433), (408, 440, 141.22), (441, 442, 17.732), (441, 285, 154.503), (441, 443, 104.12100000000001), (444, 244, 6.797), (444, 445, 120.088), (444, 443, 180.93500000000003), (274, 393, 52.408), (274, 271, 117.32400000000001), (274, 238, 175.964), (274, 446, 193.923), (393, 274, 52.408), (393, 447, 61.562999999999995), (393, 391, 146.486), (448, 175, 22.376), (448, 449, 4.712), (448, 450, 39.099000000000004), (343, 342, 23.313000000000002), (343, 340, 17.057000000000002), (343, 450, 60.066), (352, 447, 72.285), (352, 351, 210.209), (352, 354, 62.387), (447, 352, 72.285), (447, 392, 155.59199999999998), (447, 393, 61.562999999999995), (350, 351, 6.729), (350, 349, 55.647), (350, 392, 102.169), (392, 391, 76.78), (392, 350, 102.16900000000001), (392, 447, 155.59199999999998), (273, 451, 163.899), (273, 275, 32.449), (273, 271, 24.719), (451, 452, 73.155), (451, 273, 163.899), (451, 453, 113.733), (451, 454, 93.327), (451, 455, 76.69), (362, 386, 83.019), (362, 355, 155.851), (362, 361, 24.48), (282, 456, 42.161), (282, 281, 177.212), (282, 457, 66.83), (282, 458, 175.965), (282, 280, 128.473), (280, 279, 134.018), (280, 458, 121.386), (280, 282, 128.473), (280, 278, 64.962), (456, 459, 64.727), (456, 282, 42.161), (456, 457, 52.383), (455, 451, 76.69), (329, 460, 137.197), (329, 327, 59.976), (329, 333, 34.444), (460, 329, 137.197), (460, 461, 22.158), (460, 462, 154.439), (32, 463, 35.716), (32, 464, 394.54), (32, 28, 198.102), (32, 465, 95.202), (462, 466, 5.409), (462, 338, 32.691), (462, 460, 154.43899999999996), (462, 467, 65.033), (335, 334, 67.154), (335, 333, 37.866), (335, 332, 2.409), (161, 160, 37.727), (161, 158, 52.531), (161, 334, 55.741), (334, 158, 75.808), (334, 333, 52.588), (334, 161, 55.741), (334, 335, 67.154), (334, 162, 82.518), (162, 334, 82.518), (162, 160, 21.16), (162, 164, 16.32), (323, 317, 15.758), (323, 321, 15.744), (323, 468, 116.831), (469, 468, 78.225), (469, 468, 98.877), (469, 321, 26.524), (321, 314, 32.134), (321, 323, 15.744), (321, 469, 26.524), (373, 470, 22.926), (373, 369, 164.231), (373, 372, 226.566), (470, 373, 22.926), (470, 471, 3.553), (470, 472, 35.942), (473, 474, 113.604), (473, 472, 17.466), (473, 475, 20.681), (475, 476, 21.271), (475, 473, 20.681), (475, 471, 22.871), (476, 477, 165.165), (476, 475, 21.271), (476, 478, 87.299), (479, 480, 6.039), (479, 481, 56.443), (479, 482, 6.02), (479, 483, 65.453), (484, 485, 4.738), (484, 486, 255.8), (484, 487, 102.572), (484, 488, 140.392), (487, 489, 49.486), (487, 490, 68.484), (487, 491, 47.335), (487, 484, 102.572), (487, 485, 100.797), (492, 493, 14.682), (492, 494, 13.45), (492, 495, 5.571), (401, 496, 19.418), (401, 497, 106.171), (401, 399, 89.187), (497, 498, 69.944), (497, 499, 134.443), (497, 500, 93.512), (497, 401, 106.171), (501, 502, 24.181), (501, 503, 29.523), (501, 504, 75.292), (502, 505, 53.567), (502, 506, 93.715), (502, 501, 24.181), (502, 507, 90.99000000000001), (395, 394, 13.298), (395, 506, 152.688), (395, 419, 19.101), (508, 509, 74.467), (508, 417, 95.922), (508, 510, 69.77099999999999), (508, 511, 68.162), (508, 512, 90.609), (508, 513, 72.104), (514, 515, 256.885), (514, 516, 298.298), (514, 517, 289.955), (514, 512, 9.998), (518, 457, 48.626), (518, 519, 89.939), (518, 520, 82.467), (518, 521, 30.186), (516, 458, 140.358), (516, 522, 72.563), (516, 514, 298.298), (516, 523, 167.153), (516, 524, 267.318), (516, 525, 81.547), (526, 522, 29.062), (526, 527, 84.963), (526, 528, 57.805), (526, 529, 143.25), (530, 531, 94.894), (530, 532, 75.899), (530, 533, 102.376), (530, 534, 92.753), (535, 536, 170.191), (535, 277, 75.371), (535, 537, 59.61), (535, 538, 143.037), (535, 539, 46.952), (387, 359, 105.244), (387, 386, 100.426), (387, 537, 211.832), (387, 540, 113.327), (359, 387, 105.244), (359, 357, 16.241999999999997), (359, 361, 15.248999999999999), (363, 360, 26.355), (363, 378, 170.2), (363, 377, 132.966), (363, 366, 166.431), (378, 541, 336.691), (378, 363, 170.2), (378, 375, 12.82), (541, 378, 336.691), (541, 542, 5.495), (541, 358, 15.496), (543, 369, 323.007), (543, 370, 2.613), (543, 544, 13.358), (545, 546, 362.619), (545, 547, 7.394), (545, 544, 5.689), (383, 381, 321.034), (383, 548, 9.175), (383, 549, 16.523), (549, 550, 4.941), (549, 385, 325.608), (549, 383, 16.523), (544, 472, 375.295), (544, 545, 5.689), (544, 543, 13.358), (472, 544, 375.295), (472, 470, 35.942), (472, 473, 17.466), (481, 477, 118.24), (481, 479, 56.443), (481, 551, 77.453), (486, 484, 255.8), (486, 551, 54.998), (486, 552, 126.109), (553, 552, 98.693), (553, 554, 146.462), (553, 478, 177.837), (552, 489, 196.281), (552, 553, 98.693), (552, 486, 126.109), (555, 556, 17.662), (555, 557, 10.965), (558, 485, 136.027), (558, 490, 52.214), (558, 557, 437.49), (558, 559, 469.13800000000003), (490, 558, 52.214), (490, 487, 68.484), (490, 560, 5.059), (561, 562, 165.572), (561, 563, 6.908), (561, 564, 91.854), (561, 565, 6.103), (561, 566, 177.40800000000002), (561, 567, 163.966), (491, 560, 86.956), (491, 568, 168.981), (491, 489, 60.169), (491, 487, 47.335), (569, 570, 176.91), (569, 571, 218.21), (569, 572, 234.778), (569, 563, 218.977), (573, 574, 173.251), (573, 570, 5.218), (573, 564, 66.508), (573, 575, 159.10800000000003), (574, 576, 48.779), (574, 573, 173.251), (574, 577, 170.591), (574, 578, 69.296), (494, 579, 7.53), (494, 580, 9.514), (494, 492, 13.450000000000001), (581, 577, 134.012), (581, 582, 14.413), (581, 583, 13.811000000000002), (582, 575, 133.854), (582, 584, 13.442000000000002), (582, 581, 14.413), (584, 572, 136.5), (584, 585, 11.556000000000001), (584, 582, 13.442), (586, 587, 139.3), (586, 588, 11.093999999999998), (586, 589, 8.361), (588, 590, 138.866), (588, 591, 14.121), (588, 586, 11.094), (592, 593, 135.453), (592, 594, 15.003), (592, 595, 13.954), (594, 596, 133.245), (594, 583, 11.342), (594, 592, 15.002999999999998), (597, 598, 142.471), (597, 587, 100.493), (597, 599, 152.36), (597, 600, 139.075), (601, 602, 162.186), (601, 603, 47.119), (601, 598, 80.871), (604, 605, 77.132), (604, 606, 7.324), (604, 607, 46.328), (605, 608, 64.605), (605, 604, 77.132), (605, 609, 58.206), (610, 608, 6.185), (610, 611, 204.472), (610, 612, 16.815), (610, 613, 55.431), (613, 609, 27.009), (613, 614, 93.499), (613, 610, 55.431), (613, 615, 98.049), (286, 285, 103.234), (286, 616, 100.891), (286, 284, 58.151999999999994), (285, 521, 137.421), (285, 286, 103.234), (285, 441, 154.503), (285, 283, 112.045), (515, 514, 256.885), (515, 617, 195.15), (515, 614, 11.988), (618, 619, 83.568), (618, 620, 125.558), (618, 554, 52.975), (618, 621, 68.226), (618, 567, 93.01), (554, 618, 52.975), (554, 553, 146.462), (554, 619, 64.571), (554, 621, 43.612), (621, 474, 187.649), (621, 618, 68.226), (621, 554, 43.612), (621, 622, 167.333), (623, 624, 8.637), (623, 580, 9.152000000000001), (623, 625, 13.706999999999999), (489, 487, 49.486), (489, 626, 98.497), (489, 552, 196.281), (489, 491, 60.169), (489, 627, 211.38), (595, 628, 135.97), (595, 592, 13.954), (595, 591, 9.89), (536, 535, 170.191), (536, 629, 75.222), (536, 382, 159.753), (531, 530, 94.894), (531, 629, 138.918), (531, 534, 53.521), (531, 624, 221.357), (585, 630, 138.864), (585, 589, 13.445999999999998), (585, 584, 11.556000000000001), (589, 631, 138.534), (589, 586, 8.361), (589, 585, 13.446000000000002), (591, 632, 137.827), (591, 595, 9.89), (591, 588, 14.120999999999999), (583, 633, 132.349), (583, 581, 13.811), (583, 594, 11.341999999999999), (547, 622, 327.455), (547, 545, 7.394), (547, 548, 16.588), (548, 634, 315.251), (548, 383, 9.175), (548, 547, 16.588), (358, 357, 394.367), (358, 550, 2.347), (358, 541, 15.496), (379, 371, 193.376), (379, 370, 18.055999999999997), (379, 542, 14.051000000000002), (580, 635, 10.889), (580, 623, 9.152), (580, 494, 9.514), (495, 636, 14.013), (495, 492, 5.571), (495, 637, 30.691), (637, 384, 15.28), (637, 495, 30.691), (637, 625, 7.41), (625, 638, 16.403), (625, 637, 7.41), (625, 623, 13.707), (512, 639, 89.645), (512, 524, 95.602), (512, 617, 104.084), (512, 514, 9.998), (512, 508, 90.609), (520, 518, 82.467), (520, 640, 41.267), (520, 509, 71.783), (520, 459, 102.995), (459, 524, 94.249), (459, 456, 64.727), (459, 458, 194.95), (459, 520, 102.995), (458, 516, 140.358), (458, 280, 121.386), (458, 282, 175.965), (458, 523, 121.451), (458, 459, 194.95), (509, 513, 12.93), (509, 520, 71.783), (509, 524, 93.846), (509, 508, 74.467), (421, 419, 16.072000000000003), (421, 418, 14.13), (421, 641, 144.779), (642, 643, 104.664), (642, 639, 83.82), (642, 644, 101.156), (642, 645, 98.379), (418, 646, 142.311), (418, 421, 14.13), (418, 416, 17.078000000000003), (647, 426, 235.352), (647, 429, 188.865), (647, 648, 34.883), (426, 647, 235.352), (426, 427, 44.467), (426, 422, 15.184000000000001), (432, 431, 83.222), (432, 434, 154.528), (432, 649, 39.376), (429, 649, 34.886), (429, 647, 188.865), (429, 427, 89.78), (434, 433, 196.815), (434, 650, 83.747), (434, 432, 154.528), (217, 651, 156.808), (217, 249, 131.475), (217, 213, 31.358), (651, 217, 156.808), (651, 652, 8.776), (651, 249, 204.455), (249, 651, 204.455), (249, 217, 131.475), (249, 245, 10.057), (537, 539, 38.557), (537, 535, 59.61), (537, 387, 211.832), (537, 385, 186.845), (277, 539, 56.827), (277, 535, 75.37100000000001), (277, 275, 243.149), (277, 278, 157.346), (385, 540, 65.499), (385, 549, 325.608), (385, 381, 229.178), (385, 537, 186.845), (376, 371, 243.40099999999998), (376, 375, 73.353), (376, 542, 288.244), (369, 373, 164.231), (369, 368, 57.266), (369, 543, 323.007), (382, 638, 59.529), (382, 381, 52.592), (382, 536, 159.753), (653, 654, 122.585), (653, 493, 69.623), (653, 634, 59.069), (634, 653, 59.069), (634, 636, 73.544), (634, 548, 315.251), (634, 622, 225.269), (622, 547, 327.455), (622, 621, 167.333), (622, 546, 80.119), (622, 634, 225.269), (474, 473, 113.604), (474, 621, 187.649), (474, 478, 112.109), (474, 546, 76.716), (478, 476, 87.299), (478, 474, 112.109), (478, 553, 177.837), (546, 545, 362.619), (546, 474, 76.716), (546, 622, 80.119), (471, 470, 3.553), (471, 475, 22.871000000000002), (471, 655, 277.63599999999997), (655, 656, 4.773), (655, 471, 277.63599999999997), (655, 657, 101.781), (655, 374, 95.07300000000001), (477, 481, 118.24), (477, 476, 165.165), (477, 657, 151.566), (657, 483, 25.25), (657, 477, 151.566), (657, 658, 6.001), (657, 655, 101.781), (551, 486, 54.998), (551, 481, 77.453), (551, 488, 165.39000000000001), (488, 484, 140.392), (488, 480, 138.64100000000002), (488, 551, 165.39000000000001), (620, 618, 125.558), (620, 654, 64.708), (620, 659, 87.978), (571, 630, 150.91), (571, 569, 218.21), (571, 660, 179.215), (571, 661, 163.89200000000002), (661, 565, 504.545), (661, 662, 253.367), (661, 571, 163.89200000000002), (599, 597, 152.36), (599, 660, 23.119), (599, 631, 165.876), (602, 600, 76.31), (602, 663, 121.478), (602, 664, 7.072), (602, 601, 162.186), (632, 591, 137.827), (632, 590, 85.967), (632, 628, 73.848), (628, 595, 135.97), (628, 665, 177.061), (628, 593, 95.24300000000001), (628, 632, 73.84800000000001), (593, 592, 135.453), (593, 532, 148.082), (593, 596, 86.899), (593, 628, 95.24300000000001), (596, 594, 133.245), (596, 534, 121.562), (596, 633, 77.437), (596, 593, 86.899), (633, 666, 128.151), (633, 583, 132.349), (633, 577, 84.211), (633, 596, 77.437), (577, 581, 134.012), (577, 574, 170.591), (577, 575, 85.68199999999999), (577, 633, 84.21099999999998), (575, 582, 133.854), (575, 667, 3.589), (575, 573, 159.10800000000003), (575, 577, 85.68200000000002), (572, 584, 136.5), (572, 569, 234.778), (572, 630, 95.369), (572, 667, 71.553), (630, 571, 150.91), (630, 585, 138.864), (630, 631, 85.731), (630, 572, 95.36900000000001), (631, 589, 138.534), (631, 599, 165.876), (631, 587, 76.52499999999999), (631, 630, 85.731), (587, 597, 100.493), (587, 586, 139.3), (587, 590, 84.44300000000001), (587, 631, 76.525), (590, 588, 138.866), (590, 600, 111.477), (590, 632, 85.967), (590, 587, 84.44300000000001), (614, 613, 93.499), (614, 668, 162.61), (614, 515, 11.988), (614, 615, 100.22999999999999), (669, 498, 73.284), (669, 670, 143.761), (669, 500, 28.288), (377, 366, 108.991), (377, 363, 132.966), (377, 375, 112.381), (389, 355, 82.796), (389, 446, 23.84), (389, 367, 102.537), (278, 523, 155.292), (278, 276, 145.637), (278, 280, 64.962), (278, 277, 157.346), (671, 538, 82.668), (671, 672, 19.494), (671, 523, 86.463), (671, 528, 34.494), (533, 530, 102.376), (533, 527, 20.357), (533, 532, 59.273), (533, 673, 15.373), (532, 674, 164.608), (532, 530, 75.899), (532, 593, 148.082), (532, 533, 59.273), (524, 512, 95.602), (524, 459, 94.249), (524, 516, 267.318), (524, 509, 93.846), (576, 579, 192.873), (576, 574, 48.779), (576, 578, 62.879), (576, 666, 143.592), (666, 635, 175.514), (666, 633, 128.151), (666, 576, 143.592), (666, 534, 132.925), (578, 574, 69.296), (578, 659, 77.797), (578, 566, 104.709), (578, 576, 62.879), (578, 564, 222.957), (534, 596, 121.562), (534, 531, 53.521), (534, 666, 132.925), (534, 530, 92.753), (675, 676, 33.331), (675, 677, 30.162), (675, 678, 42.261), (675, 679, 75.896), (639, 642, 83.82), (639, 512, 89.645), (639, 617, 36.802), (639, 510, 92.658), (680, 681, 15.519), (680, 682, 92.061), (680, 670, 257.906), (600, 597, 139.075), (600, 590, 111.477), (600, 602, 76.31), (598, 597, 142.471), (598, 601, 80.871), (598, 683, 116.77799999999999), (598, 684, 57.886), (683, 660, 78.992), (683, 684, 118.238), (683, 598, 116.77799999999999), (679, 677, 56.305), (679, 685, 55.023), (679, 675, 75.896), (679, 686, 48.396), (677, 675, 30.162), (677, 643, 118.28), (677, 687, 74.595), (677, 679, 56.305), (499, 496, 126.873), (499, 688, 189.06), (499, 689, 86.705), (499, 497, 134.443), (496, 499, 126.873), (496, 403, 6.555), (496, 401, 19.418), (496, 690, 153.18), (405, 691, 115.892), (405, 406, 7.454), (405, 692, 169.746), (405, 402, 92.291), (691, 405, 115.892), (691, 404, 97.898), (691, 693, 67.498), (691, 694, 227.235), (692, 695, 6.738), (692, 694, 142.691), (692, 425, 179.351), (692, 405, 169.746), (690, 696, 5.681), (690, 689, 118.893), (690, 496, 153.18), (690, 697, 178.579), (694, 692, 142.691), (694, 698, 37.8), (694, 648, 180.311), (694, 691, 227.235), (698, 694, 37.8), (689, 688, 157.492), (689, 690, 118.893), (689, 503, 180.19), (689, 499, 86.705), (699, 700, 48.976), (699, 688, 20.807), (699, 701, 11.417), (699, 681, 80.17), (678, 676, 25.854), (678, 700, 18.084), (678, 702, 13.755), (678, 675, 42.261), (703, 648, 94.296), (648, 425, 143.536), (648, 703, 94.296), (648, 647, 34.883), (648, 694, 180.311), (425, 648, 143.536), (425, 704, 7.264), (425, 692, 179.351), (425, 422, 231.503), (697, 705, 6.463), (697, 504, 77.045), (697, 690, 178.579), (697, 396, 92.313), (503, 505, 56.201), (503, 504, 67.139), (503, 501, 29.523), (503, 689, 180.19), (643, 677, 118.28), (643, 642, 104.664), (643, 706, 112.467), (643, 686, 94.542), (643, 707, 93.187), (504, 503, 67.139), (504, 697, 77.045), (504, 501, 75.292), (507, 420, 94.158), (507, 706, 70.926), (507, 708, 61.975), (507, 502, 90.99000000000001), (708, 644, 28.525), (708, 641, 93.785), (708, 511, 58.016), (708, 507, 61.975), (511, 646, 94.138), (511, 510, 8.393), (511, 708, 58.016), (511, 508, 68.162), (513, 509, 12.93), (513, 415, 94.937), (513, 640, 62.641), (513, 508, 72.104), (640, 520, 41.267), (640, 414, 94.156), (640, 519, 57.408), (640, 513, 62.641), (519, 518, 89.939), (519, 411, 93.438), (519, 709, 77.94), (519, 640, 57.408), (709, 410, 92.916), (709, 442, 45.434), (709, 710, 96.006), (709, 519, 77.94), (498, 670, 113.541), (498, 497, 69.944), (498, 669, 73.284), (498, 681, 225.794), (670, 498, 113.541), (670, 669, 143.761), (670, 680, 257.906), (500, 497, 93.512), (500, 669, 28.288), (500, 400, 38.797), (404, 691, 97.898), (404, 402, 158.757), (404, 693, 107.648), (686, 679, 48.396), (686, 668, 87.697), (686, 643, 94.542), (686, 685, 22.708), (685, 679, 55.023), (685, 711, 51.067), (685, 686, 22.708), (685, 712, 54.859), (566, 567, 111.064), (566, 578, 104.709), (566, 659, 78.957), (566, 561, 177.40800000000002), (659, 578, 77.797), (659, 620, 87.978), (659, 579, 239.768), (659, 566, 78.957), (624, 623, 8.637), (624, 629, 214.38799999999998), (624, 635, 14.931), (624, 531, 221.357), (629, 531, 138.918), (629, 536, 75.222), (629, 673, 241.644), (629, 672, 251.07799999999997), (629, 624, 214.38799999999998), (672, 528, 29.011), (672, 673, 86.499), (672, 671, 19.494), (672, 629, 251.078), (528, 672, 29.011), (528, 526, 57.805), (528, 522, 71.736), (528, 671, 34.494), (522, 526, 29.062), (522, 516, 72.563), (522, 525, 87.025), (522, 528, 71.736), (525, 713, 83.114), (525, 516, 81.547), (525, 517, 92.57600000000001), (525, 522, 87.025), (517, 714, 57.459), (517, 514, 289.955), (517, 615, 68.235), (517, 525, 92.57600000000001), (615, 613, 98.049), (615, 517, 68.235), (615, 614, 100.23), (615, 714, 87.458), (715, 716, 9.213), (715, 717, 830.982), (715, 559, 723.0), (612, 608, 17.991), (612, 718, 203.391), (612, 610, 16.815), (612, 663, 236.321), (609, 608, 42.529), (609, 605, 58.206), (609, 613, 27.009), (609, 714, 168.79600000000002), (608, 609, 42.529), (608, 612, 17.991), (608, 610, 6.185), (608, 605, 64.605), (719, 718, 38.258), (719, 720, 53.125), (719, 721, 83.717), (719, 722, 127.631), (663, 602, 121.478), (663, 722, 89.82900000000001), (663, 612, 236.32099999999997), (663, 723, 192.466), (722, 663, 89.82900000000001), (722, 719, 127.631), (722, 716, 525.919), (722, 718, 133.84199999999998), (716, 715, 9.213), (716, 662, 72.749), (716, 722, 525.919), (662, 716, 72.749), (662, 660, 123.63), (662, 661, 253.367), (662, 724, 267.51800000000003), (660, 599, 23.119), (660, 683, 78.992), (660, 662, 123.63), (660, 571, 179.215), (684, 683, 118.238), (684, 603, 75.193), (684, 598, 57.886), (724, 603, 4.488), (724, 723, 13.574), (724, 662, 267.51800000000003), (721, 719, 83.717), (721, 725, 129.493), (721, 712, 158.393), (721, 720, 100.066), (717, 720, 90.81), (717, 715, 830.982), (717, 682, 370.82800000000003), (720, 719, 53.125), (720, 721, 100.066), (720, 717, 90.81), (676, 726, 52.271), (676, 675, 33.331), (676, 725, 146.244), (676, 678, 25.854), (682, 726, 35.112), (682, 680, 92.061), (682, 717, 370.828), (681, 699, 80.17), (681, 680, 15.519), (681, 498, 225.794), (681, 702, 89.237), (707, 505, 94.096), (707, 706, 63.958), (707, 727, 115.359), (707, 643, 93.187), (706, 644, 80.985), (706, 707, 63.958), (706, 507, 70.926), (706, 643, 112.467), (644, 510, 62.796), (644, 706, 80.985), (644, 642, 101.156), (644, 708, 28.525), (510, 644, 62.796), (510, 639, 92.658), (510, 511, 8.393), (510, 508, 69.77099999999999), (611, 711, 32.497), (611, 610, 204.472), (611, 728, 41.715), (611, 718, 223.505), (668, 728, 13.612), (668, 686, 87.697), (668, 614, 162.61), (668, 645, 69.013), (645, 617, 124.661), (645, 668, 69.013), (645, 642, 98.379), (641, 708, 93.785), (641, 421, 144.779), (641, 420, 41.506), (641, 646, 41.473), (420, 419, 147.991), (420, 507, 94.158), (420, 506, 62.07000000000001), (420, 641, 41.506), (506, 395, 152.688), (506, 502, 93.715), (506, 396, 114.634), (506, 420, 62.07), (396, 697, 92.313), (396, 394, 140.45), (396, 506, 114.63399999999999), (646, 511, 94.138), (646, 418, 142.311), (646, 641, 41.473), (646, 417, 47.315), (649, 429, 34.886), (649, 432, 39.376), (649, 316, 73.388), (316, 315, 40.932), (316, 295, 84.982), (316, 649, 73.388), (729, 730, 174.587), (731, 730, 207.274), (559, 715, 723.0), (559, 558, 469.13800000000003), (320, 300, 29.423), (320, 310, 27.1), (320, 90, 87.221), (90, 88, 31.682000000000002), (90, 104, 143.1), (90, 320, 87.221), (650, 652, 68.129), (650, 434, 83.747), (650, 248, 140.417), (248, 650, 140.417), (248, 732, 5.835), (248, 435, 235.68), (248, 245, 232.14700000000002), (242, 240, 231.981), (242, 733, 6.034), (242, 710, 72.622), (242, 440, 93.572), (710, 242, 72.622), (710, 442, 112.386), (710, 709, 96.006), (442, 709, 45.434), (442, 441, 17.732), (442, 710, 112.386), (442, 521, 106.585), (521, 518, 30.186), (521, 285, 137.421), (521, 442, 106.585), (521, 457, 38.807), (457, 282, 66.83), (457, 518, 48.626), (457, 521, 38.807), (457, 456, 52.383), (417, 416, 139.828), (417, 508, 95.922), (417, 646, 47.315), (417, 415, 49.871), (414, 640, 94.156), (414, 412, 144.33), (414, 411, 38.489000000000004), (414, 415, 43.936), (410, 709, 92.916), (410, 407, 151.945), (410, 411, 53.256), (410, 440, 108.128), (415, 513, 94.937), (415, 413, 141.824), (415, 417, 49.871), (415, 414, 43.936), (411, 519, 93.438), (411, 409, 147.811), (411, 414, 38.489000000000004), (411, 410, 53.25600000000001), (523, 278, 155.292), (523, 516, 167.153), (523, 538, 80.461), (523, 671, 86.463), (523, 458, 121.451), (538, 671, 82.668), (538, 523, 80.461), (538, 535, 143.037), (288, 205, 17.227), (288, 287, 25.253), (288, 652, 65.205), (652, 651, 8.776), (652, 650, 68.129), (652, 288, 65.205), (505, 502, 53.567), (505, 727, 148.315), (505, 707, 94.096), (505, 503, 56.201), (727, 505, 148.315), (727, 687, 46.409), (727, 707, 115.359), (727, 701, 51.776), (687, 727, 46.409), (687, 700, 35.385), (687, 701, 32.253), (687, 677, 74.595), (700, 687, 35.385), (700, 702, 12.599), (700, 678, 18.084), (700, 699, 48.976), (702, 700, 12.599), (702, 726, 55.437), (702, 681, 89.237), (702, 678, 13.755), (726, 682, 35.112), (726, 676, 52.271), (726, 702, 55.437), (230, 224, 3.762), (230, 734, 3.689), (230, 735, 5.192), (230, 227, 175.889), (440, 242, 93.572), (440, 410, 108.128), (440, 408, 141.22), (37, 36, 34.317), (37, 35, 57.084), (37, 6, 61.248000000000005), (736, 737, 198.375), (736, 737, 175.378), (736, 738, 200.77200000000002), (737, 739, 8.39), (737, 736, 198.375), (737, 736, 175.378), (739, 737, 8.39), (739, 740, 13.319), (739, 741, 41.766), (740, 738, 38.211), (740, 739, 13.319), (740, 742, 30.539), (741, 742, 14.792), (741, 743, 12.57), (741, 739, 41.766), (743, 741, 12.57), (743, 744, 17.057), (743, 745, 182.245), (742, 741, 14.792), (742, 746, 20.985), (742, 740, 30.539), (744, 743, 17.057), (744, 746, 14.989), (744, 747, 29.554000000000002), (738, 748, 13.905), (738, 740, 38.211), (738, 736, 200.772), (738, 463, 59.691), (746, 744, 14.989), (746, 749, 19.927), (746, 742, 20.985), (747, 750, 19.965), (747, 751, 134.96200000000002), (747, 744, 29.554000000000002), (748, 738, 13.905), (748, 752, 59.842), (748, 753, 70.452), (750, 747, 19.965), (750, 754, 89.072), (750, 755, 92.68), (749, 753, 19.135), (749, 756, 100.417), (749, 746, 19.927), (463, 32, 35.716), (463, 738, 59.691), (463, 757, 68.083), (463, 464, 400.456), (753, 749, 19.135), (753, 752, 26.15), (753, 758, 82.88000000000001), (753, 748, 70.452), (745, 759, 22.196), (745, 760, 27.604), (745, 743, 182.24499999999998), (759, 745, 22.196), (752, 748, 59.842), (752, 761, 46.75), (752, 753, 26.15), (760, 751, 3.786), (760, 762, 24.592), (760, 745, 27.604), (751, 760, 3.786), (751, 763, 18.21), (751, 747, 134.962), (763, 755, 8.544), (763, 751, 18.21), (763, 764, 39.065), (755, 763, 8.544), (755, 754, 12.856), (755, 750, 92.68), (762, 760, 24.592), (754, 755, 12.856), (754, 765, 2.883), (754, 750, 89.072), (765, 754, 2.883), (765, 756, 25.971), (765, 766, 52.977000000000004), (761, 767, 40.841), (761, 752, 46.75), (761, 730, 360.15), (756, 765, 25.971), (756, 749, 100.417), (756, 758, 28.692999999999998), (756, 768, 73.295), (767, 758, 3.74), (767, 761, 40.841), (767, 769, 106.63), (758, 767, 3.74), (758, 756, 28.692999999999998), (758, 753, 82.88000000000001), (764, 763, 39.065), (766, 765, 52.977000000000004), (768, 756, 73.295), (769, 767, 106.63), (693, 770, 82.926), (693, 691, 67.498), (693, 404, 107.648), (693, 771, 40.821), (152, 157, 34.609), (152, 159, 7.064), (152, 151, 13.106), (159, 157, 34.306), (159, 154, 13.907), (159, 152, 7.064), (163, 154, 40.548), (163, 772, 11.74), (163, 773, 10.401), (229, 227, 9.121), (229, 232, 32.514), (229, 774, 23.889000000000003), (229, 735, 171.81), (232, 228, 8.168), (232, 236, 95.164), (232, 229, 32.514), (232, 775, 24.507), (775, 241, 4.672), (775, 776, 187.006), (775, 774, 31.992), (775, 232, 24.507), (244, 444, 6.797), (244, 240, 6.102), (244, 777, 94.417), (774, 246, 5.251), (774, 775, 31.992), (774, 778, 170.22), (774, 229, 23.889000000000003), (779, 397, 54.026), (247, 245, 5.822), (247, 243, 19.097), (247, 732, 232.13), (243, 733, 231.85), (243, 240, 6.852), (243, 247, 19.097), (732, 248, 5.835), (732, 733, 19.174), (732, 437, 235.511), (732, 247, 232.13), (733, 732, 19.174), (733, 242, 6.034), (733, 439, 234.966), (733, 243, 231.85), (430, 436, 59.926), (430, 431, 2.559), (430, 428, 19.383000000000003), (437, 732, 235.511), (437, 780, 25.601), (437, 439, 19.327), (437, 435, 5.729), (436, 435, 25.484), (436, 423, 18.3), (436, 780, 6.049), (436, 430, 59.926), (439, 781, 25.971), (439, 733, 234.966), (439, 408, 6.433), (439, 437, 19.327), (780, 437, 25.601), (780, 782, 18.342), (780, 781, 19.097), (780, 436, 6.049), (423, 436, 18.3), (423, 422, 25.191), (423, 782, 6.101), (423, 428, 59.468), (782, 780, 18.342), (782, 424, 25.07), (782, 783, 18.993), (782, 423, 6.101), (781, 783, 18.491), (781, 439, 25.971), (781, 438, 6.412), (781, 780, 19.097), (438, 397, 18.539), (438, 408, 26.085), (438, 784, 54.073), (438, 781, 6.412), (783, 398, 24.707), (783, 781, 18.491), (783, 397, 6.344), (783, 782, 18.993), (424, 782, 25.07), (424, 704, 231.651), (424, 422, 6.471), (424, 398, 18.626), (397, 394, 24.591), (397, 438, 18.539), (397, 779, 54.026), (397, 783, 6.344), (398, 705, 232.459), (398, 783, 24.707), (398, 424, 18.626), (398, 394, 6.352), (784, 438, 54.073), (704, 705, 18.999), (704, 425, 7.264), (704, 424, 231.651), (704, 695, 179.296), (705, 697, 6.463), (705, 704, 18.999), (705, 696, 178.758), (705, 398, 232.459), (695, 696, 19.078), (695, 692, 6.738), (695, 704, 179.296), (695, 406, 166.172), (696, 690, 5.681), (696, 695, 19.078), (696, 403, 156.271), (696, 705, 178.758), (406, 405, 7.454), (406, 403, 21.292), (406, 695, 166.172), (406, 402, 96.013), (403, 406, 21.292), (403, 496, 6.555), (403, 696, 156.271), (403, 399, 106.634), (166, 165, 51.077), (166, 341, 30.792), (166, 170, 35.58), (785, 449, 80.626), (449, 448, 4.712), (449, 786, 30.781), (449, 785, 80.626), (215, 214, 4.676), (215, 213, 5.38), (215, 223, 28.504), (306, 307, 47.196999999999996), (306, 304, 18.65), (306, 305, 15.898000000000001), (305, 304, 4.151), (305, 307, 30.686), (305, 306, 15.898000000000001), (254, 255, 48.264), (254, 4, 6.634), (254, 201, 14.674), (257, 255, 10.26), (257, 261, 8.787), (257, 787, 37.491), (257, 201, 55.795), (261, 184, 37.652), (261, 257, 8.787), (261, 259, 11.271), (261, 263, 60.764), (787, 184, 8.232), (787, 183, 14.746), (787, 257, 37.491), (184, 261, 37.652), (184, 182, 14.24), (184, 787, 8.232), (184, 258, 11.747), (185, 211, 25.911), (185, 260, 13.689), (185, 182, 35.290000000000006), (179, 178, 20.995), (179, 180, 3.091), (179, 191, 6.945), (180, 226, 8.736), (180, 179, 3.091), (180, 178, 21.35), (190, 192, 4.487), (190, 186, 10.373000000000001), (190, 189, 9.289), (226, 180, 8.736), (226, 225, 29.342999999999996), (226, 221, 28.976), (211, 185, 25.911), (211, 269, 10.294), (211, 193, 7.606), (269, 211, 10.294), (269, 268, 17.425), (269, 267, 8.769), (188, 187, 15.602), (188, 212, 11.543), (188, 225, 9.259), (188, 186, 6.229), (225, 220, 11.677), (225, 221, 4.101), (225, 188, 9.259), (225, 226, 29.343), (267, 210, 10.303), (267, 268, 14.762), (267, 265, 9.351), (267, 269, 8.769), (212, 188, 11.543), (212, 268, 8.821), (212, 220, 6.58), (212, 193, 25.103), (220, 218, 4.946), (220, 268, 9.543), (220, 225, 11.677), (220, 212, 6.58), (210, 267, 10.303), (210, 208, 11.335), (210, 204, 27.11), (210, 266, 12.353), (207, 222, 5.755), (207, 206, 10.492), (207, 223, 6.685), (207, 218, 7.984), (224, 230, 3.762), (224, 734, 3.621), (224, 219, 28.674999999999997), (734, 230, 3.689), (734, 735, 2.574), (734, 224, 3.621), (734, 788, 29.296), (204, 203, 6.762), (204, 208, 26.168999999999997), (204, 210, 27.11), (204, 266, 38.49), (788, 214, 5.959), (788, 778, 3.475), (788, 734, 29.296), (214, 788, 5.959), (214, 213, 4.232), (214, 216, 2.318), (214, 215, 4.676), (246, 774, 5.251), (246, 245, 7.284), (246, 241, 31.973), (246, 216, 171.304), (241, 240, 8.083), (241, 775, 4.672), (241, 246, 31.973), (241, 789, 187.69600000000003), (790, 236, 85.653), (791, 776, 18.839), (776, 775, 187.006), (776, 791, 18.839), (776, 789, 2.665), (776, 272, 158.227), (789, 776, 2.665), (789, 616, 5.367), (789, 241, 187.69600000000003), (616, 789, 5.367), (616, 445, 56.067), (616, 286, 100.891), (272, 284, 7.12), (272, 270, 28.659), (272, 776, 158.227), (777, 244, 94.417), (770, 693, 82.926), (688, 499, 189.06), (688, 701, 23.199), (688, 699, 20.807), (688, 689, 157.492), (701, 688, 23.199), (701, 687, 32.253), (701, 727, 51.776), (701, 699, 11.417), (400, 500, 38.797), (400, 399, 119.743), (728, 711, 45.976), (728, 668, 13.612), (728, 611, 41.715), (711, 685, 51.067), (711, 611, 32.497), (711, 728, 45.976), (711, 712, 53.985), (712, 711, 53.985), (712, 725, 87.906), (712, 685, 54.859), (712, 721, 158.393), (725, 721, 129.493), (725, 676, 146.244), (725, 712, 87.906), (718, 612, 203.391), (718, 719, 38.258), (718, 611, 223.505), (718, 722, 133.84199999999998), (617, 512, 104.084), (617, 515, 195.15), (617, 639, 36.802), (617, 645, 124.661), (673, 629, 241.644), (673, 527, 15.607), (673, 533, 15.373), (673, 672, 86.499), (527, 533, 20.357), (527, 526, 84.963), (527, 673, 15.607), (527, 529, 184.182), (529, 527, 184.182), (529, 526, 143.25), (529, 713, 14.926), (713, 529, 14.926), (713, 525, 83.114), (713, 714, 24.803), (792, 793, 32.063), (792, 714, 17.705), (792, 606, 172.67600000000002), (792, 794, 49.912), (730, 729, 174.587), (730, 795, 124.08500000000001), (730, 731, 207.274), (730, 761, 360.15), (714, 792, 17.705), (714, 517, 57.459), (714, 609, 168.79600000000002), (714, 674, 70.17999999999999), (714, 713, 24.803), (714, 615, 87.458), (773, 796, 12.021), (773, 150, 3.932), (773, 163, 10.401), (87, 72, 191.632), (87, 73, 48.187), (87, 797, 13.685), (797, 798, 20.451999999999998), (797, 799, 12.401), (797, 87, 13.685), (799, 800, 32.718), (799, 800, 37.330999999999996), (799, 797, 12.401), (30, 33, 59.312), (30, 33, 357.02299999999997), (30, 28, 107.923), (33, 30, 59.312), (33, 30, 357.02299999999997), (33, 27, 68.33800000000001), (801, 129, 24.34), (801, 802, 52.526), (801, 128, 50.036), (802, 801, 52.526), (355, 389, 82.796), (355, 367, 81.134), (355, 362, 155.851), (355, 386, 120.322), (355, 354, 169.916), (539, 277, 56.827), (539, 537, 38.557), (539, 535, 46.952), (539, 454, 159.995), (452, 388, 74.919), (452, 451, 73.155), (452, 454, 71.914), (452, 453, 74.445), (446, 453, 70.541), (446, 389, 23.84), (446, 274, 193.923), (453, 452, 74.445), (453, 446, 70.541), (453, 451, 113.733), (453, 388, 96.502), (388, 452, 74.919), (388, 454, 111.358), (388, 453, 96.502), (388, 386, 71.745), (454, 452, 71.914), (454, 451, 93.327), (454, 388, 111.358), (454, 539, 159.995), (238, 270, 62.236999999999995), (238, 237, 115.562), (238, 274, 175.964), (567, 627, 155.11), (567, 566, 111.064), (567, 618, 93.01), (567, 561, 163.966), (567, 562, 177.396), (562, 568, 105.772), (562, 561, 165.572), (562, 567, 177.396), (209, 208, 6.34), (209, 268, 6.631), (209, 222, 6.897), (222, 207, 5.755), (222, 209, 6.897), (222, 268, 4.02), (222, 218, 10.025), (196, 803, 7.644), (196, 195, 7.688), (196, 200, 20.014), (196, 197, 10.198), (202, 804, 6.552), (202, 803, 4.319), (202, 199, 18.055), (138, 137, 121.037), (138, 134, 122.94099999999999), (138, 805, 55.1), (138, 806, 79.376), (144, 31, 119.34599999999998), (144, 23, 65.725), (144, 807, 192.333), (808, 82, 8.844), (808, 809, 23.517999999999997), (808, 810, 6.965), (811, 82, 10.017), (811, 83, 31.342), (811, 24, 67.662), (800, 812, 8.732), (800, 799, 37.331), (800, 799, 32.717999999999996), (798, 797, 20.452), (798, 813, 11.291), (798, 812, 27.875999999999998), (812, 800, 8.732), (812, 810, 8.719), (812, 798, 27.875999999999998), (813, 810, 25.213), (813, 809, 7.913), (813, 798, 11.291), (810, 813, 25.213), (810, 812, 8.719), (810, 808, 6.965), (809, 813, 7.913), (809, 808, 23.517999999999997), (809, 82, 34.061), (75, 54, 34.806), (75, 46, 108.99600000000001), (75, 81, 115.99300000000001), (54, 45, 95.439), (54, 75, 34.806), (54, 55, 72.049), (54, 83, 132.34500000000003), (83, 811, 31.342), (83, 81, 19.891000000000002), (83, 54, 132.345), (82, 198, 14.075), (82, 81, 5.048), (82, 811, 10.017), (82, 808, 8.844), (82, 809, 34.061), (198, 82, 14.075), (198, 195, 150.965), (198, 85, 38.736), (263, 200, 6.671), (263, 201, 9.106), (263, 262, 13.696), (263, 261, 60.764), (201, 254, 14.674), (201, 263, 9.106), (201, 199, 6.362), (201, 257, 55.795), (200, 264, 14.381), (200, 199, 9.275), (200, 263, 6.671), (200, 196, 20.014), (200, 803, 17.238), (2, 1, 7.684), (2, 195, 9.812), (2, 804, 12.068), (2, 814, 69.178), (804, 202, 6.552), (804, 2, 12.068), (804, 803, 4.711), (803, 196, 7.644), (803, 804, 4.711), (803, 202, 4.319), (803, 200, 17.238), (468, 469, 78.225), (468, 469, 98.877), (468, 323, 116.831), (815, 665, 7.623), (815, 793, 17.474), (815, 816, 93.165), (606, 604, 7.324), (606, 792, 172.67600000000002), (606, 816, 61.015), (816, 606, 61.015), (816, 815, 93.165), (816, 793, 107.113), (793, 815, 17.474), (793, 792, 32.063), (793, 816, 107.113), (607, 604, 46.328), (607, 817, 93.023), (607, 818, 28.46), (818, 607, 28.46), (664, 817, 33.571), (664, 817, 57.972), (664, 602, 7.072), (817, 607, 93.023), (817, 664, 33.571), (817, 664, 57.97200000000001), (794, 792, 49.912), (665, 628, 177.061), (665, 815, 7.623), (665, 674, 57.17), (674, 532, 164.608), (674, 714, 70.17999999999999), (674, 665, 57.17), (170, 169, 6.97), (170, 341, 11.663), (170, 166, 35.58), (181, 168, 11.451), (181, 330, 2.094), (181, 819, 41.631), (181, 17, 12.899), (16, 164, 6.706), (16, 15, 11.34), (16, 160, 39.001), (290, 287, 66.57400000000001), (290, 292, 61.312999999999995), (290, 205, 93.86599999999999), (205, 288, 17.227), (205, 203, 120.768), (205, 290, 93.866), (251, 820, 16.011), (251, 250, 82.09200000000001), (251, 821, 113.69500000000001), (821, 251, 113.69500000000001), (820, 251, 16.011), (820, 822, 151.21), (820, 303, 75.765), (822, 820, 151.21), (619, 626, 171.489), (619, 618, 83.568), (619, 627, 84.586), (619, 554, 64.571), (627, 568, 119.271), (627, 619, 84.586), (627, 489, 211.38), (627, 567, 155.11), (568, 491, 168.981), (568, 562, 105.772), (568, 627, 119.271), (823, 626, 97.22200000000001), (626, 489, 98.497), (626, 619, 171.489), (626, 823, 97.22200000000001), (638, 382, 59.529), (638, 625, 16.403), (638, 384, 17.836), (384, 381, 63.767), (384, 637, 15.28), (384, 638, 17.836), (384, 636, 59.250000000000014), (636, 634, 73.544), (636, 495, 14.013), (636, 384, 59.24999999999999), (636, 493, 15.256), (493, 653, 69.623), (493, 492, 14.682), (493, 636, 15.256), (579, 494, 7.53), (579, 576, 192.873), (579, 659, 239.768), (579, 635, 15.413), (635, 580, 10.889), (635, 666, 175.514), (635, 624, 14.931), (635, 579, 15.413), (771, 693, 40.821), (153, 824, 10.268), (153, 156, 9.928), (153, 151, 7.018000000000001), (156, 146, 14.112), (156, 153, 9.928), (156, 825, 11.585999999999999), (824, 153, 10.268), (824, 825, 6.013), (824, 826, 27.759), (826, 827, 12.079), (826, 828, 90.273), (826, 824, 27.759), (825, 824, 6.013), (825, 156, 11.585999999999999), (825, 827, 27.223999999999997), (380, 796, 6.051), (380, 772, 6.138), (380, 364, 130.414), (796, 380, 6.051), (796, 773, 12.021), (796, 772, 3.398), (772, 380, 6.138), (772, 796, 3.398), (772, 163, 11.74), (53, 51, 1.311), (53, 45, 8.879), (53, 55, 96.25399999999999), (183, 182, 6.746), (183, 194, 24.103), (183, 787, 14.746), (183, 256, 12.022), (194, 183, 24.103), (194, 189, 38.048), (194, 193, 36.99), (234, 237, 37.256), (234, 233, 37.004), (234, 239, 34.09), (239, 237, 17.684), (239, 234, 34.09), (239, 829, 32.278), (239, 830, 16.77), (829, 831, 57.669999999999995), (829, 831, 65.049), (829, 239, 32.278), (831, 829, 65.049), (831, 829, 57.669999999999995), (831, 832, 42.237), (832, 831, 42.237), (833, 834, 14.552), (833, 835, 3.144), (833, 836, 61.958999999999996), (834, 833, 14.552), (837, 838, 28.939), (837, 839, 22.404999999999998), (837, 830, 19.748), (839, 837, 22.404999999999998), (840, 830, 27.638), (830, 840, 27.638), (830, 837, 19.748), (830, 239, 16.77), (836, 833, 61.959), (836, 838, 18.654), (836, 838, 21.643), (835, 833, 3.144), (838, 837, 28.939), (838, 836, 18.654), (838, 836, 21.643), (150, 773, 3.932), (150, 149, 13.542000000000002), (150, 148, 11.545), (51, 53, 1.311), (51, 49, 12.589), (51, 52, 18.546999999999997), (253, 841, 44.936), (253, 842, 8.671), (253, 250, 234.41200000000003), (841, 253, 44.936), (841, 842, 95.912), (843, 844, 11.261), (844, 845, 0.416), (844, 846, 12.744), (844, 843, 11.261), (847, 848, 11.249), (846, 24, 5.752), (846, 814, 13.686), (846, 844, 12.744), (848, 847, 11.249), (848, 849, 0.637), (848, 814, 12.864), (850, 85, 10.999), (85, 850, 10.999), (85, 198, 38.736), (85, 84, 94.19600000000001), (125, 105, 6.272), (125, 303, 37.172000000000004), (445, 444, 120.088), (445, 851, 61.748), (445, 616, 56.06699999999999), (851, 445, 61.748), (852, 853, 13.057), (852, 854, 58.979), (852, 64, 154.152), (853, 852, 13.057), (853, 855, 81.22), (853, 854, 69.961), (854, 852, 58.979), (854, 853, 69.961), (854, 856, 71.17399999999999), (855, 853, 81.22), (805, 856, 3.834), (805, 857, 27.147), (805, 138, 55.1), (858, 64, 4.895), (858, 857, 6.315), (858, 66, 66.951), (859, 654, 41.715), (654, 620, 64.708), (654, 653, 122.585), (654, 859, 41.715), (540, 550, 352.333), (540, 385, 65.499), (540, 387, 113.327), (550, 540, 352.333), (550, 358, 2.347), (550, 549, 4.941), (345, 860, 7.878), (345, 342, 27.91), (345, 861, 47.295), (862, 860, 32.501), (860, 345, 7.878), (860, 862, 32.501000000000005), (860, 863, 39.092), (786, 449, 30.781), (786, 863, 43.383), (786, 390, 66.084), (861, 345, 47.295), (390, 864, 31.62), (390, 786, 66.084), (390, 347, 60.723), (865, 864, 8.501), (864, 865, 8.501), (864, 390, 31.62), (864, 866, 14.058), (866, 864, 14.058), (863, 786, 43.383), (863, 867, 9.463), (863, 860, 39.092), (867, 863, 9.463), (126, 118, 18.834), (126, 868, 64.914), (126, 137, 33.143), (868, 126, 64.914), (127, 79, 43.034), (795, 730, 124.08500000000001), (129, 801, 24.34), (129, 869, 44.315), (129, 78, 51.602000000000004), (869, 129, 44.315), (128, 78, 33.936), (128, 124, 372.62899999999996), (128, 801, 50.036), (326, 7, 131.011), (34, 8, 20.532), (34, 870, 41.322), (34, 11, 99.71100000000001), (870, 34, 41.322), (40, 171, 4.834), (40, 11, 42.217), (40, 12, 62.989000000000004), (12, 176, 36.236), (12, 10, 35.389), (12, 40, 62.989000000000004), (171, 169, 28.194), (171, 40, 4.834), (171, 819, 122.346), (819, 181, 41.631), (819, 171, 122.346), (819, 339, 89.67500000000001), (339, 461, 82.454), (339, 819, 89.67500000000001), (339, 337, 158.55), (461, 339, 82.454), (461, 460, 22.158), (461, 871, 66.327), (871, 461, 66.327), (467, 462, 65.033), (807, 144, 192.33300000000003), (443, 872, 49.82), (443, 444, 180.93500000000003), (443, 441, 104.12100000000001), (872, 443, 49.82), (564, 573, 66.508), (564, 561, 91.854), (564, 578, 222.957), (175, 448, 22.376), (175, 873, 27.954), (175, 174, 44.853), (873, 175, 27.954), (873, 874, 61.611000000000004), (873, 875, 6.963), (450, 343, 60.066), (450, 448, 39.099000000000004), (450, 875, 24.295), (875, 450, 24.295), (875, 873, 6.962999999999999), (875, 874, 8.202), (340, 343, 17.057000000000002), (340, 172, 34.619), (340, 876, 48.513), (876, 874, 28.154999999999998), (876, 340, 48.513000000000005), (876, 173, 48.664), (874, 876, 28.154999999999998), (874, 875, 8.202), (874, 873, 61.611), (173, 174, 61.202), (173, 169, 13.706), (173, 876, 48.664), (332, 335, 2.409), (332, 331, 50.106), (332, 828, 119.394), (845, 844, 0.416), (849, 848, 0.637), (814, 2, 69.178), (814, 846, 13.686), (814, 848, 12.864), (828, 827, 124.066), (828, 826, 90.273), (828, 332, 119.394), (842, 253, 8.671), (842, 841, 95.912), (842, 877, 114.99), (42, 878, 45.979), (42, 29, 26.243000000000002), (42, 38, 203.28500000000003), (878, 42, 45.979), (29, 879, 58.253), (29, 28, 90.594), (29, 42, 26.243000000000002), (879, 29, 58.253), (140, 131, 132.825), (17, 181, 12.899), (17, 15, 35.443999999999996), (17, 167, 11.648), (330, 181, 2.094), (330, 328, 5.412), (330, 327, 14.582), (18, 164, 4.275), (18, 15, 11.735), (18, 328, 42.961), (328, 327, 10.312), (328, 18, 42.961), (328, 330, 5.412), (370, 368, 317.885), (370, 543, 2.613), (370, 379, 18.056), (542, 541, 5.495), (542, 376, 288.244), (542, 379, 14.051000000000002), (880, 881, 41.634), (880, 880, 132.377), (880, 880, 132.377), (556, 555, 17.662), (556, 881, 42.652), (556, 882, 207.798), (881, 880, 41.634), (881, 556, 42.652), (881, 883, 105.599), (882, 884, 37.469), (882, 556, 207.798), (882, 883, 143.65300000000002), (885, 886, 28.411), (884, 882, 37.469), (884, 482, 121.663), (884, 886, 44.260999999999996), (483, 657, 25.25), (483, 887, 5.795), (483, 479, 65.453), (24, 846, 5.752), (24, 22, 208.52100000000002), (24, 41, 119.134), (24, 811, 67.662), (735, 734, 2.574), (735, 230, 5.192), (735, 229, 171.81), (735, 778, 28.235999999999997), (778, 774, 170.22), (778, 788, 3.475), (778, 216, 5.111), (778, 735, 28.235999999999997), (216, 214, 2.318), (216, 213, 5.221), (216, 778, 5.111), (216, 246, 171.304), (236, 232, 95.164), (236, 888, 6.853), (236, 790, 85.653), (236, 233, 8.744), (888, 236, 6.853), (136, 133, 11.242), (136, 142, 125.272), (136, 889, 148.667), (889, 136, 148.667), (338, 337, 54.559), (338, 462, 32.691), (338, 336, 85.918), (59, 890, 28.339000000000002), (59, 890, 20.625999999999998), (59, 60, 5.305), (59, 58, 33.41), (890, 134, 5.337000000000001), (890, 59, 28.339), (890, 59, 20.625999999999998), (890, 806, 44.282000000000004), (134, 60, 20.52), (134, 132, 16.81), (134, 890, 5.337), (134, 138, 122.94099999999999), (60, 134, 20.52), (60, 59, 5.305), (60, 132, 11.13), (60, 58, 33.705), (806, 856, 133.67999999999998), (806, 890, 44.282000000000004), (806, 138, 79.376), (98, 56, 18.632), (98, 102, 64.851), (98, 101, 72.322), (65, 66, 35.281000000000006), (65, 101, 35.254), (65, 61, 31.847), (66, 858, 66.951), (66, 65, 35.281), (66, 61, 10.728000000000002), (63, 102, 39.503), (63, 61, 33.551), (63, 101, 34.226), (102, 98, 64.851), (102, 63, 39.503), (102, 101, 11.685), (64, 858, 4.895), (64, 852, 154.152), (64, 61, 73.19500000000001), (64, 891, 6.3100000000000005), (856, 805, 3.834), (856, 854, 71.174), (856, 891, 26.861), (856, 806, 133.68), (891, 857, 4.528), (891, 64, 6.3100000000000005), (891, 856, 26.861), (857, 805, 27.147), (857, 858, 6.315), (857, 891, 4.528), (857, 892, 88.367), (892, 857, 88.367), (341, 170, 11.663), (341, 172, 17.664), (341, 166, 30.792), (827, 826, 12.079), (827, 825, 27.223999999999997), (827, 828, 124.06599999999999), (893, 894, 9.358), (893, 895, 40.577), (893, 896, 44.282000000000004), (897, 898, 24.381), (897, 899, 15.928), (897, 900, 8.014), (898, 897, 24.381), (901, 895, 38.629), (901, 902, 46.069), (901, 899, 121.06), (902, 903, 4.955), (902, 904, 17.651), (902, 901, 46.069), (905, 906, 36.025), (905, 895, 2.498), (905, 903, 24.375), (906, 905, 36.025), (906, 907, 29.596), (906, 908, 20.762), (906, 894, 21.277), (907, 906, 29.596), (907, 909, 21.615), (907, 910, 19.929), (909, 908, 29.598), (909, 911, 21.185), (909, 907, 21.615), (908, 909, 29.598), (908, 903, 22.346), (908, 912, 22.136), (908, 906, 20.762), (904, 912, 22.56), (904, 902, 17.651), (904, 899, 41.268), (911, 912, 29.631), (911, 913, 41.91), (911, 909, 21.185), (912, 904, 22.56), (912, 911, 29.631), (912, 908, 22.136), (894, 893, 9.358), (894, 906, 21.277), (894, 910, 29.578), (910, 894, 29.578), (910, 907, 19.929), (910, 896, 30.271), (896, 910, 30.271), (896, 893, 44.282000000000004), (896, 887, 57.654), (895, 893, 40.577), (895, 901, 38.629), (895, 905, 2.498), (903, 908, 22.346), (903, 905, 24.375), (903, 902, 4.955), (899, 904, 41.268), (899, 897, 15.928), (899, 913, 52.576), (899, 901, 121.06000000000002), (913, 899, 52.576), (913, 914, 24.789), (913, 911, 41.91), (915, 916, 29.965), (915, 917, 20.623), (915, 914, 21.475), (916, 915, 29.965), (916, 918, 31.041), (916, 919, 20.603), (916, 920, 20.812), (918, 916, 31.041), (918, 920, 39.684), (918, 900, 29.583), (900, 897, 8.014), (900, 919, 22.259), (900, 918, 29.583), (914, 919, 30.194), (914, 915, 21.475), (914, 913, 24.789), (919, 900, 22.259), (919, 914, 30.194), (919, 916, 20.603), (920, 916, 20.812), (920, 917, 29.716), (920, 918, 39.684), (917, 920, 29.716), (917, 921, 27.356), (917, 915, 20.623), (921, 917, 27.356), (922, 883, 35.85), (923, 924, 129.13), (925, 926, 10.742), (927, 926, 34.733), (926, 925, 10.742), (926, 927, 34.733), (926, 924, 33.817), (924, 926, 33.817), (924, 883, 35.48), (924, 923, 129.13), (883, 881, 105.599), (883, 924, 35.48), (883, 922, 35.85), (883, 882, 143.653), (928, 886, 32.389), (886, 885, 28.411), (886, 928, 32.389), (886, 884, 44.260999999999996), (155, 147, 9.152), (155, 146, 11.687999999999999), (147, 155, 9.152), (147, 145, 11.114), (466, 462, 5.409), (466, 929, 78.31700000000001), (466, 929, 81.04), (929, 464, 29.243000000000002), (929, 466, 78.31700000000001), (929, 466, 81.04), (464, 32, 394.54), (464, 929, 29.243000000000002), (464, 463, 400.45600000000013), (757, 930, 203.95899999999995), (757, 463, 68.083), (757, 465, 64.197), (465, 931, 149.143), (465, 32, 95.202), (465, 757, 64.197), (931, 930, 14.775), (931, 932, 289.63), (931, 465, 149.14300000000003), (930, 931, 14.775), (930, 757, 203.95900000000003), (930, 932, 299.96399999999994), (932, 930, 299.964), (932, 931, 289.63), (557, 555, 10.965), (557, 933, 379.777), (557, 558, 437.49), (480, 479, 6.039), (480, 933, 6.185), (480, 488, 138.641), (485, 558, 136.027), (485, 484, 4.738), (485, 933, 282.242), (485, 487, 100.797), (933, 480, 6.185), (933, 557, 379.777), (933, 482, 6.191), (933, 485, 282.242), (482, 884, 121.663), (482, 933, 6.191), (482, 479, 6.02), (482, 887, 69.138), (887, 483, 5.795), (887, 896, 57.654), (887, 658, 25.976), (887, 482, 69.13799999999999), (374, 655, 95.07300000000001), (374, 372, 48.150999999999996), (374, 656, 101.57799999999999), (656, 934, 18.105), (656, 655, 4.773), (656, 374, 101.578), (656, 658, 102.13799999999999), (934, 656, 18.105), (658, 935, 6.327), (658, 657, 6.001), (658, 656, 102.138), (658, 887, 25.976), (935, 658, 6.327), (560, 490, 5.059), (560, 491, 86.956), (560, 565, 528.748), (570, 573, 5.218), (570, 569, 176.91), (570, 667, 160.189), (570, 563, 153.94), (667, 575, 3.589), (667, 570, 160.189), (667, 572, 71.553), (563, 561, 6.908), (563, 565, 3.438), (563, 569, 218.977), (563, 570, 153.94), (565, 561, 6.103), (565, 563, 3.438), (565, 661, 504.5450000000001), (565, 560, 528.7479999999999), (23, 22, 4.266), (23, 144, 65.725), (23, 55, 86.24699999999999), (303, 125, 37.172), (303, 820, 75.765), (303, 301, 64.866), (877, 842, 114.99000000000001), (318, 298, 1.687), (318, 294, 34.589000000000006), (318, 293, 134.463), (292, 291, 3.947), (292, 260, 182.61200000000002), (292, 290, 61.312999999999995), (130, 121, 33.822), (130, 80, 70.251), (130, 80, 264.6), (723, 603, 10.78), (723, 724, 13.574), (723, 663, 192.466), (112, 110, 171.754), (112, 97, 57.211), (112, 120, 84.11000000000001), (603, 601, 47.119), (603, 724, 4.488), (603, 723, 10.78), (603, 684, 75.193)]
#big_graph_path = transform_and_find_eulerian_path(tab, False)
#print(big_graph_path)

Initial Graph: [(0, 1, 9), (1, 2, 28), (1, 22, 27), (2, 3, 6), (3, 4, 13), (3, 6, 20), (4, 5, 25), (5, 6, 11), (5, 14, 18), (6, 7, 7), (7, 8, 18), (8, 9, 13), (8, 10, 29), (10, 11, 9), (11, 12, 10), (11, 18, 7), (12, 13, 11), (12, 14, 2), (13, 14, 5), (13, 15, 5), (15, 16, 11), (15, 21, 11), (16, 17, 18), (16, 21, 19), (17, 18, 23), (17, 19, 4), (18, 19, 24), (19, 20, 9), (21, 22, 1)]
To Eulerian Graph: [(0, 1, 9), (1, 2, 28), (1, 22, 27), (2, 3, 6), (3, 4, 13), (3, 6, 20), (4, 5, 25), (5, 6, 11), (5, 14, 18), (6, 7, 7), (7, 8, 18), (8, 9, 13), (8, 10, 29), (10, 11, 9), (11, 12, 10), (11, 18, 7), (12, 13, 11), (12, 14, 2), (13, 14, 5), (13, 15, 5), (15, 16, 11), (15, 21, 11), (16, 17, 18), (16, 21, 19), (17, 18, 23), (17, 19, 4), (18, 19, 24), (19, 20, 9), (21, 22, 1), (14, 12, 2), (19, 17, 4), (15, 13, 5), (18, 11, 7), (1, 0, 9), (6, 5, 11), (9, 8, 13), (21, 16, 19)]
[*] Compute Eulerian path: 97.3 %
Eulerian path: [(3, 2), (2, 1), (1, 0), (0, 1), (1, 22), (22, 21), (21, 15), (15, 13)

Voici un test sur un graph un peu plus gros
<img src="../assets/img/big_path.png" width="500">

On obtient peut-être pas le chemin "le" plus optimisé mais en tout cas on a la garantie d'avoir un chemin efficace.